In [ ]:
"""
File Name: main.ipynb
Author: Eli Claggett
Date: Apr 2023

Description:
    In-class project implementing a convolutional neural network for face classification and verification
    
"""

In [ ]:
# Imports
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import torch.nn.functional as F
from tqdm import tqdm
from PIL import Image
import torch.nn as nn
import pandas as pd
import numpy as np
import torchvision
import wandb
import torch
import math
import glob
import os
import gc

In [7]:
# Configuration
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DATA_DIR    = './data'
TRAIN_DIR   = os.path.join(DATA_DIR, "train") 
VAL_DIR     = os.path.join(DATA_DIR, "dev")
TEST_DIR    = os.path.join(DATA_DIR, "test")

config = {
    'batch_size': 64,
    'lr': 0.08,
    'epochs': 150
}

In [8]:
# Training Configuration

# Transforms
train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.RandomAdjustSharpness(sharpness_factor=1, p = 0.2),
    torchvision.transforms.RandomHorizontalFlip(p=0.4),
])

# No transforms on validation set!!!
valid_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

In [9]:
# Create train and validation datasets
train_dataset   = torchvision.datasets.ImageFolder(TRAIN_DIR, transform= train_transforms)
valid_dataset   = torchvision.datasets.ImageFolder(VAL_DIR, transform= valid_transforms)

# Create train and validation data loaders
train_loader = torch.utils.data.DataLoader(
    dataset     = train_dataset, 
    batch_size  = config['batch_size'], 
    shuffle     = True,
    pin_memory  = True
    num_workers = 0, 
)

valid_loader = torch.utils.data.DataLoader(
    dataset     = valid_dataset, 
    batch_size  = config['batch_size'],
    shuffle     = False,
    num_workers = 0
)

In [10]:
# Create test dataset and data loader
class ClassificationTestDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transforms):
        self.data_dir   = data_dir
        self.transforms = transforms
        self.img_paths  = list(map(lambda fname: os.path.join(self.data_dir, fname), sorted(os.listdir(self.data_dir))))

    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        return self.transforms(Image.open(self.img_paths[idx]))
    
test_dataset = ClassificationTestDataset(TEST_DIR, transforms = valid_transforms)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = config['batch_size'], shuffle = False, drop_last = False, num_workers = 0)

In [12]:
# Describe data
print("Number of classes    : ", len(train_dataset.classes))
print("No. of train images  : ", train_dataset.__len__())
print("Shape of image       : ", train_dataset[0][0].shape)
print("Batch size           : ", config['batch_size'])
print("Train batches        : ", train_loader.__len__())
print("Val batches          : ", valid_loader.__len__())

Number of classes    :  7000
No. of train images  :  140000
Shape of image       :  torch.Size([3, 224, 224])
Batch size           :  64
Train batches        :  2188
Val batches          :  547


In [14]:
# Implement the convolutional blocks in ResNet

class ResNetBlock(nn.Module):
    def __init__(self, c_in, c_out, stride = 1, downsample = None):
        super(ResNetBlock, self).__init__()        

        self.relu   = nn.ReLU()
        self.conv1  = nn.Sequential(
                        nn.Conv2d(c_in, c_out, kernel_size = 1, stride = 1, padding = 0),
                        nn.BatchNorm2d(c_out)
                    )
        self.conv2  = nn.Sequential(
                        nn.Conv2d(c_out, c_out, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm2d(c_out)
                    )
        self.conv3  = nn.Sequential(
                        nn.Conv2d(c_out, c_out * 4, kernel_size = 1, stride = 1, padding = 0),
                        nn.BatchNorm2d(c_out * 4)
                    )
        self.downsample = downsample
    
    def forward(self, x):
        xOriginal = x

        output = self.conv1(x)
        output = self.relu(output)
        output = self.conv2(output)
        output = self.relu(output)
        output = self.conv3(output)
        
        if self.downsample is not None:
            xOriginal = self.downsample(xOriginal)
        output += xOriginal
        output = self.relu(output)
        return output

In [15]:
# Implement ResNet

class ResNet(nn.Module):
    def __init__(self, block, channels = 3, classes = 7000):
        super(ResNet, self).__init__()
        
        self.c_in = 64
        
        self.layer0_conv = nn.Conv2d(channels, 64, kernel_size = 7, stride = 2, padding = 3)
        self.layer0_bn = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.poolMax = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        
        self.layer1 = self.make_resnet_layer(block, 64, 3, stride = 1)
        self.layer2 = self.make_resnet_layer(block, 128, 4, stride = 2)
        self.layer3 = self.make_resnet_layer(block, 256, 23, stride = 2)
        self.layer4 = self.make_resnet_layer(block, 512, 3, stride = 2)
        
        self.poolAvg = nn.AdaptiveAvgPool2d((1, 1))
        self.cls_layer = nn.Linear(512 * 4, classes)
        
    def make_resnet_layer(self, block, c_intermediate, blocks, stride = 1, downsample = None):
        if  self.c_in != c_intermediate * 4 or stride != 1:
            downsample = nn.Sequential(
                nn.Conv2d(self.c_in, c_intermediate * 4, kernel_size = 1, stride = stride),
                nn.BatchNorm2d(c_intermediate * 4),
            )
        
        layers = []
        layers.append(block(self.c_in, c_intermediate, stride, downsample = downsample))
        self.c_in = c_intermediate * 4
        
        for i in range(1, blocks):
            layers.append(block(self.c_in, c_intermediate))

        return nn.Sequential(*layers)
    
    def forward(self, x, return_feats = False):
        x = self.layer0_conv(x)
        x = self.layer0_bn(x)
        x = self.relu(x)
        x = self.poolMax(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.poolAvg(x)
        x = x.reshape(x.shape[0], -1)
        output = self.cls_layer(x)
        
        if return_feats:
            return x
        else:
            return output

In [16]:
# Setup the model
model = ResNet(ResNetBlock)
model = model.to(DEVICE)

# Setup the training paradigm (scheduler, optimizer, loss function)
criterion =nn.CrossEntropyLoss(label_smoothing=0.05)
scaler = torch.cuda.amp.GradScaler()
optimizer = torch.optim.SGD(model.parameters(), lr=config['lr'], momentum=0.9, weight_decay=2e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, config['epochs'])

In [17]:
# Setup a training function
def train(model, dataloader, optimizer, criterion):
    
    model.train()

    batch_bar   = tqdm(total=len(dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train', ncols=5) 
    num_correct = 0
    total_loss  = 0

    for i, (images, labels) in enumerate(dataloader):
        
        # Reset gradients to zero
        optimizer.zero_grad()

        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        # Use mixed precision
        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss    = criterion(outputs, labels)

        # Calculate accuracy and loss metrics
        num_correct     += int((torch.argmax(outputs, axis=1) == labels).sum())
        total_loss      += float(loss.item())

        # Monitor progress
        batch_bar.set_postfix(
            acc         = "{:.04f}%".format(100 * num_correct / (config['batch_size']*(i + 1))),
            loss        = "{:.04f}".format(float(total_loss / (i + 1))),
            num_correct = num_correct,
            lr          = "{:.04f}".format(float(optimizer.param_groups[0]['lr']))
        )
        
        # Use gradient scaling to prevent vanishing gradients in mixed precision operations
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update() 
      
        batch_bar.update()

    batch_bar.close()

    # Return accuracy and loss metrics
    acc         = 100 * num_correct / (config['batch_size']* len(dataloader))
    total_loss  = float(total_loss / len(dataloader))
    return acc, total_loss

In [18]:
# Setup a validation function
def validate(model, dataloader, criterion):
    
    model.eval()
    
    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=False, desc='Val', ncols=5)
    num_correct = 0.0
    total_loss = 0.0

    for i, (images, labels) in enumerate(dataloader):
        
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        # Get model outputs
        with torch.inference_mode():
            outputs = model(images)
            loss = criterion(outputs, labels)

        # Calculate accuracy and loss metrics
        num_correct += int((torch.argmax(outputs, axis=1) == labels).sum())
        total_loss += float(loss.item())

        # Monitor progress
        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / (config['batch_size']*(i + 1))),
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            num_correct=num_correct)

        batch_bar.update()
        
    batch_bar.close()

    # Return accuracy and loss metrics
    acc = 100 * num_correct / (config['batch_size']* len(dataloader))
    total_loss = float(total_loss / len(dataloader))
    return acc, total_loss

In [19]:
# Let the computer rest... :)
gc.collect()
torch.cuda.empty_cache()

## Train

In [21]:
# Track training progress on Weights & Biases
wandb.login(key="REDACTED")

run = wandb.init(
   name = "resnet-101",
    reinit = True,
    project = "resnet-ablations",
    config = config
)

In [22]:
# Train the model

best = 0.0
cache = []

for epoch in range(config['epochs']):


    # Track the current learning rate
    curr_lr = float(optimizer.param_groups[0]['lr'])

    # Train the model
    train_acc, train_loss = train(model, train_loader, optimizer, criterion)
    
    # Monitor progress
    print("\nEpoch {}/{}: \nTrain Acc {:.04f}%\t Train Loss {:.04f}\t Learning Rate {:.04f}".format(
        epoch + 1,
        config['epochs'],
        train_acc,
        train_loss,
        curr_lr))
    
    # Validate model performance
    val_acc, val_loss = validate(model, valid_loader, criterion)
    scheduler.step()
    print("Val Acc {:.04f}%\t Val Loss {:.04f}".format(val_acc, val_loss))

    cache.append({"train_loss":train_loss, 'train_Acc': train_acc, 'validation_Acc':val_acc, 
               'validation_loss': val_loss, "learning_Rate": curr_lr})
    
    # Save the best model
    if val_acc >= best:
      print("Saving model")
      torch.save({
            'model_state_dict':model.state_dict(),
            'optimizer_state_dict':optimizer.state_dict(),
            'val_acc': val_acc,
            'epoch': epoch
        },
        './checkpoint.pth')
      best = val_acc
run.finish()


Epoch 1/150: 
Train Acc 0.0057%	 Train Loss 8.8687	 Learning Rate 0.0800


Val Acc 0.0257%	 Val Loss 8.8514
Saving model



Epoch 2/150: 
Train Acc 0.0179%	 Train Loss 8.8374	 Learning Rate 0.0800


Val Acc 0.0257%	 Val Loss 8.8256
Saving model



Epoch 3/150: 
Train Acc 0.0478%	 Train Loss 8.7003	 Learning Rate 0.0800


...

Val Acc 87.5371%	 Val Loss 2.5309



Epoch 149/150: 
Train Acc 99.9771%	 Train Loss 1.2642	 Learning Rate 0.0001


Val Acc 87.5600%	 Val Loss 2.5309



Epoch 150/150: 
Train Acc 99.9771%	 Train Loss 1.2643	 Learning Rate 0.0000


Val Acc 87.5686%	 Val Loss 2.5221


## Test

In [24]:
# Load the model from a saved state to run on the test set
model = ResNet(ResNetBlock)
model.load_state_dict(torch.load('checkpoint.pth')['model_state_dict'])
model = model.to(DEVICE)

In [25]:
# Setup test function
def test(model,dataloader):

  model.eval()

  batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=False, desc='Test')
  test_results = []
  
  for i, (images) in enumerate(dataloader):
      
      images = images.to(DEVICE)

      # Run inference on test set
      with torch.inference_mode():
        outputs = model(images)

      # Get results
      outputs = torch.argmax(outputs, axis=1).detach().cpu().numpy().tolist()
      test_results.extend(outputs)
      
      batch_bar.update()
  batch_bar.close()

  return test_results

In [30]:
# Get model predictions for the test set
test_results = test(model, test_loader)

In [31]:
# Save test results
with open("classification_resnet110.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(test_dataset)):
        f.write("{},{}\n".format(str(i).zfill(5) + ".jpg", test_results[i]))

## Facial Verification

Kinda like FaceID on the iPhone

In [32]:
# Get a list of known faces
known_regex = "./verification/known/*/*"
known_paths = [i.split('/')[-2] for i in sorted(glob.glob(known_regex))]

# Get some lists of unknown faces
unknown_dev_regex = "./verification/unknown_dev/*"
unknown_test_regex = "./verification/unknown_test/*"

# Load the images
unknown_dev_images = [Image.open(p) for p in tqdm(sorted(glob.glob(unknown_dev_regex)))]
unknown_test_images = [Image.open(p) for p in tqdm(sorted(glob.glob(unknown_test_regex)))]
known_images = [Image.open(p) for p in tqdm(sorted(glob.glob(known_regex)))]

# Apply transforms
transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()])

unknown_dev_images = torch.stack([transforms(x) for x in unknown_dev_images])
unknown_test_images = torch.stack([transforms(x) for x in unknown_test_images])
known_images  = torch.stack([transforms(y) for y in known_images ])

# Define the similarity metric for testing if the unknown face is the same as the known face
similarity_metric = torch.nn.CosineSimilarity(dim= 1, eps= 1e-6) 

100%|██████████| 960/960 [00:00<00:00, 4921.43it/s]


In [33]:
# Load the pretrained model
model = ResNet(ResNetBlock)
model.load_state_dict(torch.load('checkpoint.pth')['model_state_dict'])
model = model.to(DEVICE)

In [34]:
# Setup a function to test the model's performance on facial verification
def eval_verification(unknown_images, known_images, model, similarity, batch_size= config['batch_size'], mode='val'): 
    unknown_feats, known_feats = [], []

    batch_bar = tqdm(total=len(unknown_images)//batch_size, dynamic_ncols=True, position=0, leave=False, desc=mode)
    model.eval()
    
    # Extract features of the unknown faces
    for i in range(0, unknown_images.shape[0], batch_size):
        unknown_batch = unknown_images[i:i+batch_size]
        with torch.no_grad():
            unknown_feat = model(unknown_batch.float().to(DEVICE), return_feats=True)
        unknown_feats.append(unknown_feat[1])
        batch_bar.update()
    
    batch_bar.close()
    batch_bar = tqdm(total=len(known_images)//batch_size, dynamic_ncols=True, position=0, leave=False, desc=mode)
    
    # Extract features of the known faces
    for i in range(0, known_images.shape[0], batch_size):
        known_batch = known_images[i:i+batch_size] 
        with torch.no_grad():
            known_feat = model(known_batch.float().to(DEVICE), return_feats=True)
        known_feats.append(known_feat[1])
        batch_bar.update()

    batch_bar.close()

    # Concatenate the batches
    unknown_feats = torch.cat(unknown_feats, dim=0)
    known_feats = torch.cat(known_feats, dim=0)

    # Calculate how similar the unknown faces are to the known faces
    similarity_values = torch.stack([similarity(unknown_feats, known_feature) for known_feature in known_feats])

    # See if the unknown faces are in our "database" so to speak
    max_similarity_values, predictions = similarity_values.max(0)
    max_similarity_values, predictions = max_similarity_values.cpu().numpy(), predictions.cpu().numpy()

    # If the max similarity is below the threshold, we probably don't have that face in the "database"
    threshold = 0.2
    NO_CORRESPONDENCE_LABEL = 'n000000'
    pred_id_strings = []
    
    # Get the predicted face for each unknown face
    for idx, prediction in enumerate(predictions):
        if max_similarity_values[idx] < threshold:
            pred_id_strings.append(NO_CORRESPONDENCE_LABEL)
        else:
            pred_id_strings.append(known_paths[prediction])
    
    # Get the facial verification accuracy
    if mode == 'val':
      true_ids = pd.read_csv('./verification/verification_dev.csv')['label'].tolist()
      accuracy = accuracy_score(pred_id_strings, true_ids)
      print("Verification Accuracy = {}".format(accuracy))
    
    return pred_id_strings

In [35]:
# Let the computer rest
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Run facial verification on the validation set
pred_id_strings = eval_verification(unknown_dev_images, known_images, model, similarity_metric, config['batch_size'], mode='val')

In [34]:
# Run facial verification on the test set
pred_id_strings = eval_verification(unknown_test_images, known_images, model, similarity_metric, config['batch_size'], mode='test')

In [35]:
# Record the test set results
with open("verification_resnet110.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(pred_id_strings)):
        f.write("{},{}\n".format(i, pred_id_strings[i]))